# Creating Azure Resources



In [2]:
from azure.mgmt.resource import ResourceManagementClient
from azure.common.credentials import ServicePrincipalCredentials


# Set resources' parameters
SUBSCRIPTION_NAME = 'Team Danielle Internal'
SUBSCRIPTION_ID = 'edf507a2-6235-46c5-b560-fd463ba2e771'
SP_CLIENT_ID = '095d4aa7-a8d5-4f24-aa96-c631e3dee97c'
SP_SECRET = 'a9b478dc-dbdb-403f-bd6d-1dee6c0b1bd4'
SP_TENANT = '72f988bf-86f1-41af-91ab-2d7cd011db47'

RESOURCE_GROUP = 'baimm'
LOCATION = 'eastus'
ACR_NAME = 'baimmacr'
SCHED_DOCKER_IMG = 'baimmschedimg'
BAI_DOCKER_IMG = 'baimmimg'

BAI_CLUSTER_NAME = 'baimmcluster'
BLOB_ACCOUNT = ''
BLOB_KEY = ''
BLOB_CONTAINER = ''

In [1]:
# Login to subscription
!az login
!az account set -s {SUBSCRIPTION_NAME}

ERROR: Unexpected polling state code_expired
ERROR: The subscription of '{subscription_name}' doesn't exist in cloud 'AzureCloud'.


In [3]:
# Create credentials
credentials = ServicePrincipalCredentials(client_id=SP_CLIENT_ID,
                                          secret=SP_SECRET,
                                          tenant=SP_TENANT)
# Create Resource Group
rm_client = ResourceManagementClient(credentials, SUBSCRIPTION_ID)
rm_client.resource_groups.create_or_update(RESOURCE_GROUP, {'location': LOCATION})

In [ ]:

# Create ACR
!az acr login --name {ACR_NAME}
!az acr create --resource-group bleikbaimm --name baimmacr --sku Basic

# Create Batch AI cluster
!az batchai cluster create -l {LOCATION} -g {RESOURCE_GROUP} -n {BAI_CLUSTER_NAME} -s Standard_NC6 -i UbuntuDSVM --min 2 --max 2 -u demoUser -p demoPassword --storage-account-name baimmstorage --storage-account-key yCWDQMX5U6baYh/pqpEABOHpjQYK92L8IWei5ckpdgR5dT5S7TMmm2eCQUQL9Tn4TGssgf/Xjb6xT6sv+HXDFg== --container-name models 
!az batchai auto-scale -g MyResourceGroup -w MyWorkspace -n MyCluster --min 0 --max 10